# Gather Hourly Traffic Data <br>
*Author(s): Anton Olav Glad & Filip Moen Nilsen* <br>
 <br>
This Jupyter Notebook shoes step by step how to extract **hourly** data from traffic points throughout the Norwegian Road Network. <br>
The API and documentation is available at: *https://www.vegvesen.no/trafikkdata/api* <br>
The first query aims to extract relevant traffic point IDs, which is later on used as a input variable in the second queries. <br>
<br>
**NOTE:** There is only possible to get 100 observations from each volume-query. Therefore Query #2 is evenly split into two queries. <br>

Import necessary packages:

In [51]:
import os
import json
import requests
import time
import random
from tqdm import tqdm  
import pandas as pd
from datetime import datetime
# Display all columns in the df
pd.set_option('display.max_columns', None)

Set paths etc.:

In [52]:
# define the base path where the file should be saved
base_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData'
# define the file name
file_name = "all_IDs.json"
# combine the base path and file name to create a valid file path
file_path = os.path.join(base_path, file_name)

### First query (get traffic point ID's):

In [53]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

# define the GraphQL query for fetching ID's of registration points (only Europe roads)
query = """
{
  trafficRegistrationPoints(searchQuery: {roadCategoryIds: [E ,R, F], registrationFrequency: CONTINUOUS}) {
    id
  }
}
"""

# make a POST request to the API
response = requests.post(endpoint, json={"query": query})

# check the status code of the response
if response.status_code == 200:
    # if the request was successful, get the data from the response
    data = response.json()

    # open the file for writing
    with open(file_path, "w") as file:
        # write the data to the file in JSON format
        json.dump(data, file)

Store query number 1 and convert ID's to list for inserting in query number 2:

In [65]:
# define the file path of the JSON file
file_path = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/all_IDs.json'

# open the file for reading
with open(file_path, "r") as file:
    # load the data from the file into a Python object
    data = json.load(file)

# create an empty list for the registration point ID's
registration_point_ids = []

# iterate over the data returned by the GraphQL query
for item in data["data"]["trafficRegistrationPoints"]:
    # get the ID of the current registration point
    id = item["id"]

    # add the ID to the list of registration point ID's
    registration_point_ids.append(id)

### Second query (get traffic volume for every hour in a one-week period): <br>
*NB! This query must be run **twice** because it only possible to get 100 hours per query.* <br>
*The dataframes must therefore be merged together later on.*

In [66]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

data_1 = []
with tqdm(total=len(registration_point_ids)) as pbar:     # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
  for registration_point_id in registration_point_ids:    # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
    time.sleep(random.uniform(1, 3))

    # Change to: "2018-09-23T23:00:00+02:00"
    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-04-16T00:00:00+02:00"
        to: "2018-04-19T11:00:00+02:00"  
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_1 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_1["registration_point_id"] = registration_point_id

      data_1.append(volume_data_1)

      pbar.update(1)

100%|██████████| 3126/3126 [1:47:33<00:00,  2.06s/it]  


*Second run:*

In [67]:
# define the endpoint of the API
endpoint = 'https://www.vegvesen.no/trafikkdata/api/'

data_2 = []
with tqdm(total=len(registration_point_ids)) as pbar:     # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
  for registration_point_id in registration_point_ids:    # HUSK Å FJERNE [:300] FOR Å FÅ MED ALLE TRAFFIC POINTS
    time.sleep(random.uniform(1, 3))

    # Change to: "2018-09-23T23:00:00+02:00"
    query = """ {
  trafficData(trafficRegistrationPointId: "%s") {
    volume {
      byHour(
        from: "2018-04-19T11:00:00+02:00"
        to: "2018-04-23T00:00:00+02:00"
      ) {
        edges {
          node {
            from
            to
            total {
              volumeNumbers {
                volume
              }
              coverage {
                percentage
              }
            }
          }
        }
      }
    }
  }
}
    """ % registration_point_id

    response = requests.post(endpoint, json={"query": query})

    if response.status_code == 200:
      volume_data_2 = response.json()

      # Add the registration_point_id to the volume_data dictionary
      volume_data_2["registration_point_id"] = registration_point_id

      data_2.append(volume_data_2)

      pbar.update(1)

100%|█████████▉| 3125/3126 [1:48:20<00:02,  2.08s/it]


Store data collected from query number 2:

In [68]:
# define the path and filename for the JSON file
filename_1 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1_Apr.json'
# write the data to the JSON file
with open(filename_1, "w") as f:
    json.dump(data_1, f, indent=2) # usually indent = 4, but ok 

# define the path and filename for the JSON file
filename_2 = '/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2_Apr.json'
# write the data to the JSON file
with open(filename_2, "w") as f:
    json.dump(data_2, f, indent=2) 

Open data collected from query number 2:

In [69]:
with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_1_Apr.json', 'r') as file:
    json_data_1 = json.load(file)

with open('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/hourlyVolume_2_Apr.json', 'r') as file:
    json_data_2 = json.load(file)

### def flatten_json <br>
*Takes the data gathered from query 2 (stored as a json-file), flattening the data and converts it to a pandas dataframe.*

In [70]:
def flatten_json(json_data):
    flat_list = []
    for item in json_data:
        edges = item.get("data", {}).get("trafficData", {}).get("volume", {}).get("byHour", {}).get("edges", [])
        traffic_registration_point_id = item.get("registration_point_id")

        data_count = 0
        temp_list = []
        for edge in edges:
            node = edge.get("node", {})
            total = node.get("total", {})

            if total is None:
                continue

            volume_numbers = total.get("volumeNumbers") or {}
            coverage = total.get("coverage") or {}

            if volume_numbers.get("volume") is None:
                continue

            from_datetime = datetime.fromisoformat(node.get("from").replace("+02:00", ""))
            to_datetime = datetime.fromisoformat(node.get("to").replace("+02:00", ""))

            flat_dict = {
                "traffic_point_id": traffic_registration_point_id,
                "year": from_datetime.year,
                "month": from_datetime.month,
                "day": from_datetime.day,
                "from_hour": from_datetime.strftime("%H:%M"),
                "to_hour": to_datetime.strftime("%H:%M"),
                "volume": volume_numbers.get("volume"),
                "coverage_percentage": coverage.get("percentage")
            }
            data_count += 1
            temp_list.append(flat_dict)

        if data_count >= 83:    # 83 er riktig tall egentlig            # NB! Denne burde egentlig være "if data_count == 168:", men det ser ut til at det kun er mulig å hente ut 100 timer på ett punkt om gangen.
                                            # data_count == 168 fordi 24 timer i 7 dager (24*7) = 168
            flat_list.extend(temp_list)

    return flat_list


**Read warning in function above.**

Run function, check output:

In [71]:
flat_data_1 = flatten_json(json_data_1)
flat_data_2 = flatten_json(json_data_2)
# Convert flattened data to DataFrame
volume_df_1 = pd.DataFrame(flat_data_1)
volume_df_2 = pd.DataFrame(flat_data_2)
display(volume_df_1)#[:50]
display(volume_df_2)#[:50]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,4,16,00:00,01:00,11,100.0
1,65823V1668921,2018,4,16,01:00,02:00,0,100.0
2,65823V1668921,2018,4,16,02:00,03:00,1,100.0
3,65823V1668921,2018,4,16,03:00,04:00,2,100.0
4,65823V1668921,2018,4,16,04:00,05:00,7,100.0
...,...,...,...,...,...,...,...,...
154375,10795V320297,2018,4,19,06:00,07:00,452,100.0
154376,10795V320297,2018,4,19,07:00,08:00,948,100.0
154377,10795V320297,2018,4,19,08:00,09:00,834,100.0
154378,10795V320297,2018,4,19,09:00,10:00,543,100.0


,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,4,19,11:00,12:00,38,100.0
1,65823V1668921,2018,4,19,12:00,13:00,47,100.0
2,65823V1668921,2018,4,19,13:00,14:00,43,100.0
3,65823V1668921,2018,4,19,14:00,15:00,45,100.0
4,65823V1668921,2018,4,19,15:00,16:00,45,100.0
...,...,...,...,...,...,...,...,...
159860,10795V320297,2018,4,22,19:00,20:00,558,100.0
159861,10795V320297,2018,4,22,20:00,21:00,474,100.0
159862,10795V320297,2018,4,22,21:00,22:00,317,100.0
159863,10795V320297,2018,4,22,22:00,23:00,222,100.0


### Merge the two dataframes:

In [72]:
# Concatenate the two dataframes
merged_df = pd.concat([volume_df_1, volume_df_2], ignore_index=True)

# Group by traffic_point_id and filter based on the number of data points (168 hours)
completeVolume_df = merged_df.groupby('traffic_point_id').filter(lambda x: len(x) >= 166)

# Reset the index of the filtered dataframe
completeVolume_df.reset_index(drop=True, inplace=True)


Check the merged dataframe - looks good:

In [73]:
completeVolume_df#[:60]

,traffic_point_id,year,month,day,from_hour,to_hour,volume,coverage_percentage
0,65823V1668921,2018,4,16,00:00,01:00,11,100.0
1,65823V1668921,2018,4,16,01:00,02:00,0,100.0
2,65823V1668921,2018,4,16,02:00,03:00,1,100.0
3,65823V1668921,2018,4,16,03:00,04:00,2,100.0
4,65823V1668921,2018,4,16,04:00,05:00,7,100.0
...,...,...,...,...,...,...,...,...
309101,10795V320297,2018,4,22,19:00,20:00,558,100.0
309102,10795V320297,2018,4,22,20:00,21:00,474,100.0
309103,10795V320297,2018,4,22,21:00,22:00,317,100.0
309104,10795V320297,2018,4,22,22:00,23:00,222,100.0


Save dataframe as CSV-file in directory:

In [74]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_Apr.csv"

# Convert the dataframe to a CSV file and save it to the specified path
completeVolume_df.to_csv(output_file_path, index=False)


In [75]:
print(completeVolume_df['traffic_point_id'].nunique() / len(registration_point_ids))

0.5886116442738324


In [76]:
#completeVolume_df = pd.read_csv('/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_July.csv')
#completeVolume_df

### Pivot the dataframe <br> 
*This is to display hourly volume on the y-axis (not sure if necessary)*

In [77]:
# Create a new column 'timestamp' combining year, month, day, and from_hour
completeVolume_df['timestamp'] = completeVolume_df['year'].astype(str) + '-' + completeVolume_df['month'].astype(str).str.zfill(2) + '-' + completeVolume_df['day'].astype(str).str.zfill(2) + ' ' + completeVolume_df['from_hour']

# Pivot the DataFrame using 'traffic_point_id' as index and 'timestamp' as columns
pivoted_df = completeVolume_df.pivot_table(index='traffic_point_id', columns='timestamp', values='volume')

# Reset index and fill missing values with 0 (if any)
pivoted_df.reset_index(inplace=True)
pivoted_df.fillna(0, inplace=True)

# Display pivoted dataframe
pivoted_df

timestamp,traffic_point_id,2018-04-16 00:00,2018-04-16 01:00,2018-04-16 02:00,2018-04-16 03:00,2018-04-16 04:00,2018-04-16 05:00,2018-04-16 06:00,2018-04-16 07:00,2018-04-16 08:00,2018-04-16 09:00,2018-04-16 10:00,2018-04-16 11:00,2018-04-16 12:00,2018-04-16 13:00,2018-04-16 14:00,2018-04-16 15:00,2018-04-16 16:00,2018-04-16 17:00,2018-04-16 18:00,2018-04-16 19:00,2018-04-16 20:00,2018-04-16 21:00,2018-04-16 22:00,2018-04-16 23:00,2018-04-17 00:00,2018-04-17 01:00,2018-04-17 02:00,2018-04-17 03:00,2018-04-17 04:00,2018-04-17 05:00,2018-04-17 06:00,2018-04-17 07:00,2018-04-17 08:00,2018-04-17 09:00,2018-04-17 10:00,2018-04-17 11:00,2018-04-17 12:00,2018-04-17 13:00,2018-04-17 14:00,2018-04-17 15:00,2018-04-17 16:00,2018-04-17 17:00,2018-04-17 18:00,2018-04-17 19:00,2018-04-17 20:00,2018-04-17 21:00,2018-04-17 22:00,2018-04-17 23:00,2018-04-18 00:00,2018-04-18 01:00,2018-04-18 02:00,2018-04-18 03:00,2018-04-18 04:00,2018-04-18 05:00,2018-04-18 06:00,2018-04-18 07:00,2018-04-18 08:00,2018-04-18 09:00,2018-04-18 10:00,2018-04-18 11:00,2018-04-18 12:00,2018-04-18 13:00,2018-04-18 14:00,2018-04-18 15:00,2018-04-18 16:00,2018-04-18 17:00,2018-04-18 18:00,2018-04-18 19:00,2018-04-18 20:00,2018-04-18 21:00,2018-04-18 22:00,2018-04-18 23:00,2018-04-19 00:00,2018-04-19 01:00,2018-04-19 02:00,2018-04-19 03:00,2018-04-19 04:00,2018-04-19 05:00,2018-04-19 06:00,2018-04-19 07:00,2018-04-19 08:00,2018-04-19 09:00,2018-04-19 10:00,2018-04-19 11:00,2018-04-19 12:00,2018-04-19 13:00,2018-04-19 14:00,2018-04-19 15:00,2018-04-19 16:00,2018-04-19 17:00,2018-04-19 18:00,2018-04-19 19:00,2018-04-19 20:00,2018-04-19 21:00,2018-04-19 22:00,2018-04-19 23:00,2018-04-20 00:00,2018-04-20 01:00,2018-04-20 02:00,2018-04-20 03:00,2018-04-20 04:00,2018-04-20 05:00,2018-04-20 06:00,2018-04-20 07:00,2018-04-20 08:00,2018-04-20 09:00,2018-04-20 10:00,2018-04-20 11:00,2018-04-20 12:00,2018-04-20 13:00,2018-04-20 14:00,2018-04-20 15:00,2018-04-20 16:00,2018-04-20 17:00,2018-04-20 18:00,2018-04-20 19:00,2018-04-20 20:00,2018-04-20 21:00,2018-04-20 22:00,2018-04-20 23:00,2018-04-21 00:00,2018-04-21 01:00,2018-04-21 02:00,2018-04-21 03:00,2018-04-21 04:00,2018-04-21 05:00,2018-04-21 06:00,2018-04-21 07:00,2018-04-21 08:00,2018-04-21 09:00,2018-04-21 10:00,2018-04-21 11:00,2018-04-21 12:00,2018-04-21 13:00,2018-04-21 14:00,2018-04-21 15:00,2018-04-21 16:00,2018-04-21 17:00,2018-04-21 18:00,2018-04-21 19:00,2018-04-21 20:00,2018-04-21 21:00,2018-04-21 22:00,2018-04-21 23:00,2018-04-22 00:00,2018-04-22 01:00,2018-04-22 02:00,2018-04-22 03:00,2018-04-22 04:00,2018-04-22 05:00,2018-04-22 06:00,2018-04-22 07:00,2018-04-22 08:00,2018-04-22 09:00,2018-04-22 10:00,2018-04-22 11:00,2018-04-22 12:00,2018-04-22 13:00,2018-04-22 14:00,2018-04-22 15:00,2018-04-22 16:00,2018-04-22 17:00,2018-04-22 18:00,2018-04-22 19:00,2018-04-22 20:00,2018-04-22 21:00,2018-04-22 22:00,2018-04-22 23:00
0,00000V1702725,8.0,2.0,1.0,5.0,5.0,25.0,143.0,492.0,455.0,510.0,721.0,848.0,888.0,887.0,998.0,1266.0,1374.0,1067.0,893.0,726.0,576.0,277.0,36.0,15.0,6.0,4.0,0.0,4.0,2.0,22.0,131.0,461.0,413.0,480.0,689.0,771.0,859.0,888.0,987.0,1225.0,1214.0,949.0,939.0,753.0,545.0,282.0,48.0,31.0,6.0,1.0,3.0,1.0,7.0,17.0,133.0,421.0,434.0,474.0,684.0,810.0,847.0,878.0,956.0,1190.0,1248.0,1044.0,952.0,770.0,621.0,313.0,48.0,22.0,7.0,3.0,3.0,2.0,4.0,27.0,147.0,420.0,448.0,473.0,680.0,844.0,904.0,992.0,1070.0,1213.0,1343.0,1073.0,990.0,925.0,724.0,365.0,62.0,24.0,9.0,4.0,0.0,4.0,5.0,19.0,135.0,455.0,424.0,548.0,850.0,1030.0,1072.0,1065.0,1300.0,1460.0,1466.0,1062.0,875.0,697.0,563.0,269.0,60.0,23.0,20.0,11.0,6.0,4.0,0.0,3.0,13.0,39.0,84.0,377.0,987.0,1373.0,1506.0,1441.0,1453.0,1339.0,1166.0,1063.0,696.0,404.0,105.0,52.0,27.0,32.0,28.0,7.0,2.0,2.0,1.0,0.0,0.0,12.0,33.0,76.0,147.0,251.0,362.0,452.0,394.0,362.0,226.0,165.0,121.0,82.0,82.0,50.0,34.0,15.0
1,00000V1702751,113.0,160.0,48.0,55.0,139.0,583.0,1818.0,3288.0,3152.0,2645.0,2670.0,2902.0,2892.0,3095.0,3338.0,4586.0,4149.0,3048.0,2424.0,1843.0,1610.0,1052.0,576.0,2

Store new pivoted dataframe:

In [78]:
# Specify the file path and name
output_file_path = "/Users/filipmoennilsen/Documents/Skolearbeid/BI_Master_BAN/4. Semester/Master_Thesis/Master_Python/roadNetwork_Venv/PythonFiles/trafficAPI/2_TrafficOutputData/completeAllHourlyVolume_Apr_pivoted.csv"

# Convert the dataframe to a CSV file and save it to the specified path
pivoted_df.to_csv(output_file_path, index=False)